In [223]:
import numpy as np
import random 
## b3hash

In [224]:
def okbithash(val,mod=0,mod2=0,mod3=0):
    mod = f'|{mod}|{mod2}|{mod3}|'
    s = val + mod
    h = hash(s)
    return(h%2)

In [225]:
byte_count = np.array([0, 1, 1, 2, 1, 2, 2, 3, 1, 2, 2, 3, 2, 3, 3, 4, 1, 2, 2, 3, 2, 3, 3, 4, 2, 3, 3, 4, 3, 4, 4, 5, 1, 2, 2, 3, 2, 3, 3, 4, 2, 3, 3, 4, 3, 4, 4, 5, 2, 3, 3, 4, 3, 4, 4, 5, 3, 4, 4, 5, 4, 5, 5, 6, 1, 2, 2, 3, 2, 3, 3, 4, 2, 3, 3, 4, 3, 4, 4, 5, 2, 3, 3, 4, 3, 4, 4, 5, 3, 4, 4, 5, 4, 5, 5, 6, 2, 3, 3, 4, 3, 4, 4, 5, 3, 4, 4, 5, 4, 5, 5, 6, 3, 4, 4, 5, 4, 5, 5, 6, 4, 5, 5, 6, 5, 6, 6, 7, 1, 2, 2, 3, 2, 3, 3, 4, 2, 3, 3, 4, 3, 4, 4, 5, 2, 3, 3, 4, 3, 4, 4, 5, 3, 4, 4, 5, 4, 5, 5, 6, 2, 3, 3, 4, 3, 4, 4, 5, 3, 4, 4, 5, 4, 5, 5, 6, 3, 4, 4, 5, 4, 5, 5, 6, 4, 5, 5, 6, 5, 6, 6, 7, 2, 3, 3, 4, 3, 4, 4, 5, 3, 4, 4, 5, 4, 5, 5, 6, 3, 4, 4, 5, 4, 5, 5, 6, 4, 5, 5, 6, 5, 6, 6, 7, 3, 4, 4, 5, 4, 5, 5, 6, 4, 5, 5, 6, 5, 6, 6, 7, 4, 5, 5, 6, 5, 6, 6, 7, 5, 6, 6, 7, 6, 7, 7, 8],dtype=np.uint8)

bitmasks = list(range(10))
bitmasks[0] = 0b00000001
bitmasks[1] = 0b00000010
bitmasks[2] = 0b00000100
bitmasks[3] = 0b00001000
bitmasks[4] = 0b00010000
bitmasks[5] = 0b00100000
bitmasks[6] = 0b01000000
bitmasks[7] = 0b10000000

class gbits:

    def __init__(self,gb):
        self.gb = gb
        self.num_bits = int(len(gb)*8)

    def __next__(self):
        if self.iter_idx >= self.num_bits:
            raise StopIteration()
        out = self.gb._getbit(self.iter_idx)
        self.iter_idx += 1
        return(out)

    def __getitem__(self,bit_idx):
        out = self.gb._getbit(bit_idx)
        return(out)

    def __iter__(self):
        self.iter_idx = 0
        return(self)
    
    def __len__(self):
        return(self.num_bits)

    def __repr__(self):
        s = f'<gbits>'
        return(s)


class gbytes(np.ndarray):
    def __new__(cls,b):
        buf = np.frombuffer(b,dtype=np.uint8)
        qa = np.ndarray.__new__(cls,len(b),dtype=np.uint8)
        qa[:] = buf
        return(qa)
    
    def __repr__(self):
        s = 'g\''
        for b in self:
            s += '\\x{:02x}'.format(b)
        s += '\''
        return(s)

    def _getbit(self,bit_idx):
        byte_idx = bit_idx // 8
        bit_idx = bit_idx % 8
        byte = self[byte_idx]
        bit = byte & bitmasks[bit_idx]
        bit = np.bool8(bit)
        return(bit)


    @property
    def bits(self):
        return(gbits(self))

    def bit_count(self):
        count = 0
        for b in self:
            count += byte_count[b]
        return(count)

    @staticmethod
    def similarity(gb1, gb2):
        sim_bin = gbytes(np.bitwise_not( np.bitwise_xor( gb1,gb2 ) ))
        sim = sim_bin.bit_count()
        sim = int(sim)
        return(sim)

        

In [230]:
def b3hash(seq,rounds=256):
    ''' a much bettter version '''
    out_array = np.ndarray(rounds,dtype=np.bool8)
    seq = ["<START>"] + list(seq) + ["<END>"]
    last = True
    bigram_count = dict()
    for round in range(rounds):
        sim_sum = 0
        bigram_count = dict()
        for i in range(1,len(seq)):
            bigram = (seq[i-1],seq[i])
            bigram_count.setdefault(bigram,0)
            count = bigram_count[bigram]
            h1 = okbithash(seq[i-1],round,mod2=0,mod3=count)
            h2 = okbithash(seq[i],round,mod2=1,mod3=count)
            bigram_count[bigram] += 1
            x12 = h1 ^ h2
            if x12:
                sim_sum += 1
            else:
                sim_sum -= 1
        if sim_sum > 0:
            out_array[round] = True
            last = True
        elif sim_sum < 0:
            out_array[round] = False
            last = False
        else:
            last = (not last)
            out_array[round] = last # deterministic tie-breaker
    out_array = np.packbits(out_array)
    out_array = gbytes(out_array)
    return(out_array)


In [227]:
SipHash

NameError: name 'SipHash' is not defined

In [ ]:
for _ in range(10000000):
    hashlib.blake2b(b'asdf').digest()

In [ ]:
for _ in range(10000000*8):
    hash(b'asdf')

In [233]:
def b4hash(seq):
    ''' a much bettter version '''
    rounds=512
    out_array = np.ndarray(rounds,dtype=np.bool8)
    seq = ["<START>"] + list(seq) + ["<END>"]
    last = True
    bigram_count = dict()
    hash_list = list()
    #for i in range(1,len(seq)):
    #    bigram = (seq[i-1],seq[i])
    #    bigram_count.setdefault(bigram,0)
    #    count_s = str(bigram_count[bigram])
    #    h1 = hashlib.blake2b()
#
    #    h1 = okbithash(seq[i-1],round,mod2=0,mod3=count)
    #    h2 = okbithash(seq[i],round,mod2=1,mod3=count)
    #    bigram_count[bigram] += 1
    #    x12 = h1 ^ h2
    #    if x12:
    #        sim_sum += 1
    #    else:
    #        sim_sum -= 1

    for round in range(rounds):
        sim_sum = 0
        bigram_count = dict()
        for i in range(1,len(seq)):
            bigram = seq[i-1:i+1]
            bigram_count.setdefault(bigram,0)
            count = bigram_count[bigram]
            h1 = okbithash(seq[i-1],round,mod2=0,mod3=count)
            h2 = okbithash(seq[i],round,mod2=1,mod3=count)
            bigram_count[bigram] += 1
            x12 = h1 ^ h2
            if x12:
                sim_sum += 1
            else:
                sim_sum -= 1
        if sim_sum > 0:
            out_array[round] = True
            last = True
        elif sim_sum < 0:
            out_array[round] = False
            last = False
        else:
            last = (not last)
            out_array[round] = last # deterministic tie-breaker
    out_array = np.packbits(out_array)
    out_array = gbytes(out_array)
    return(out_array)


In [ ]:
hex(0b10000000)

In [ ]:
def one_one(bit1,bit2): return(bit1 & bit2) #
def one_zero(bit1,bit2): return(bit1 & (bit1 ^ bit2))
def zero_one(bit1,bit2): return(bit2 & (bit1 ^ bit2)) 
def zero_zero(bit1,bit2): return( ~(bit1 | bit2)) 
def not_one_one(bit1,bit2): return(~(bit1 & bit2)) 
def not_one_zero(bit1,bit2): return(~(bit1 & (bit1 ^ bit2))) #
def not_zero_one(bit1,bit2): return(~(bit2 & (bit1 ^ bit2)))#
def not_zero_zero(bit1,bit2): return( (bit1 | bit2)) #
bit_funcs = np.array([one_one, one_zero, zero_one, zero_zero, not_one_one, not_one_zero, not_zero_one, not_zero_zero])

def itty_bibithash(bit1, bit2, mod1=0, mod2=0, mod3=0):
    bit1 = np.bool8(bit1)
    bit2 = np.bool8(bit2)
    idx = hash(f'|{mod1}|{mod2}|{mod3}|') % len(bit_funcs) 
    func = bit_funcs[idx]
    out = func(bit1,bit2)
    return(out)


In [ ]:
zz = np.array([0,1])
zz[np.uint8(np.bool8(2))]

In [ ]:
def itty_b3hash(b,rounds=64):
    bits = gbytes(b).bits
    bigram_count = np.array([0,0,0,0])
    out_array = np.ndarray(rounds,dtype=np.bool8)
    last = True
    for round in range(rounds):
        sim_sum = 0
        bigram_count = np.array([0,0,0,0])
        for i in range(1,len(bits)):
            b1 = bits[i-1]
            b2 = bits[i]
            bigram_ref = (b2<<1)+b1
            count = bigram_count[bigram_ref]
            h12 = itty_bibithash(b1,b2,mod1=round,mod2=count)
            bigram_count[bigram_ref] += 1
            if h12:
                sim_sum += 1
            else:
                sim_sum -= 1
        if sim_sum > 0:
            out_array[round] = True
            last = True
        elif sim_sum < 0:
            out_array[round] = False
            last = False
        else:
            last = (not last)
            out_array[round] = last # deterministic tie-breaker
    out_array = np.packbits(out_array)
    out_array = gbytes(out_array)
    return(out_array)

In [189]:
256 // 8

32

In [221]:
from collections import defaultdict
import hashlib 
import numpy as np

bitmasks = list(range(8))
bitmasks[0] = 0b00000001
bitmasks[1] = 0b00000010
bitmasks[2] = 0b00000100
bitmasks[3] = 0b00001000
bitmasks[4] = 0b00010000
bitmasks[5] = 0b00100000
bitmasks[6] = 0b01000000
bitmasks[7] = 0b10000000


def getbit(bstr,idx):
    byte_idx = idx // 8
    bit_idx = idx % 8
    val = bstr[byte_idx] & bitmasks[bit_idx]
    if val > 0:
        val = 1
    return(val)

def setbit(bstr,idx):
    byte_idx = idx // 8
    bit_idx = idx % 8
    bstr[byte_idx] |= bitmasks[bit_idx]
    #print(bstr[byte_idx])


def b5hash(input):
    # set-inclusion similarity hash
    freq = defaultdict(lambda :0) 
    bitcounts = np.zeros(256,dtype=np.int64)
    output = np.zeros(32,dtype=np.uint8)
    #for token_b in input:
    for i in range(len(input)-1):
        token_b = input[i:i+2]
        freq[token_b] += 1  # unfortunately, this uses TWO hashings 
    for token_b,count in freq.items():
        #token_b = token_b.to_bytes(1,'little') ### GET RID OF THIS JUNK
        b2b = hashlib.blake2b()
        b2b.update(token_b)
        for i in range(count):
        
            cnt_s = f'|{i}'
            cnt_b = cnt_s.encode('ascii')

            b2b2 = b2b.copy()
            b2b2.update(cnt_b)
            digest = b2b2.digest()

            for idx in range(256):
                val = getbit(digest,idx)
                if val == 1:
                    bitcounts[idx] += 1
                else:
                    bitcounts[idx] -= 1
    last = 0
    for idx in range(256):
        bcnt = bitcounts[idx]
        if bcnt > 0:
            setbit(output,idx)
        elif bcnt == 0:
            if last == 1:
                setbit(output,idx)
                last = 0
            else:
                last = 1
    output = output.tobytes()
    return(output)
    



In [236]:


def b3hash(seq,rounds=256):
    ''' a much bettter version '''
    out_array = np.ndarray(rounds,dtype=np.bool8)
    seq = ["<START>"] + list(seq) + ["<END>"]
    last = True
    bigram_count = dict()
    for round in range(rounds):
        sim_sum = 0
        bigram_count = dict()
        for i in range(1,len(seq)):
            bigram = (seq[i-1],seq[i])
            bigram_count.setdefault(bigram,0)
            count = bigram_count[bigram]
            h1 = okbithash(seq[i-1],round,mod2=0,mod3=count)
            h2 = okbithash(seq[i],round,mod2=1,mod3=count)
            bigram_count[bigram] += 1
            x12 = h1 ^ h2
            if x12:
                sim_sum += 1
            else:
                sim_sum -= 1
        if sim_sum > 0:
            out_array[round] = True
            last = True
        elif sim_sum < 0:
            out_array[round] = False
            last = False
        else:
            last = (not last)
            out_array[round] = last # deterministic tie-breaker
    out_array = np.packbits(out_array)
    out_array = gbytes(out_array)
    return(out_array)



In [238]:
for i in range(200):
    b5hash(b'aso;djfniwuaeo;nfasoiudhfaisdfnkl;jasdbnfilbasdjfkna;skljdnfl;aksjdfn;aklsjdhfpwaeiubgdfhgbyreg aerufbaweflawhjeb rri4ueyh34982729i3yr9p8hapsdifbnp39487tghbaeuriga439p8hgt43987hpt34ut9b349t8b4t39b')

In [242]:
for i in range(200):
    b3hash('aso;djfniwuaeo;nfasoiudhfaisdfnkl;jasdbnfilbasdjfkna;skljdnfl;aksjdfn;aklsjdhfpwaeiubgdfhgbyreg aerufbaweflawhjeb rri4ueyh34982729i3yr9p8hapsdifbnp39487tghbaeuriga439p8hgt43987hpt34ut9b349t8b4t39b')

In [216]:
import hashlib

In [145]:
b2b = blake2b()
b2b.update(b'sdf')


In [152]:
b2b.update(b'234')

In [153]:
b2b.hexdigest()

'5f207c779e45447e6725d8ecde425f5f5fbdf081a60878a8bc043dedb62ac9787229f26b5ac445933889e7c026b12de51c947ef36b948668bf3d66507b446b35'

In [150]:
b2b2 = b2b.copy()

In [154]:
b2b2.hexdigest()

'4bd01a53e3fa9243c57fbfcf18405a3a9b2e162d565064f9be0f96df64cb2ff65677ab222de749dd14493182c69325be20aeb749f0eadade5e0817d08834e9df'

In [66]:
from collections import defaultdict

In [168]:
zz = defaultdict(lambda :0)
zz
zz[1]+=1
zz[1]
#zz.get(1,0)

dict_items([(1, 1)])

In [170]:
for i,j in zz.items():
    print(i)
    print(j)

1
1


In [142]:
zz = defaultdict(lambda :0)
for i in range(100000000):
    zz[1]+=1

In [143]:
zz = defaultdict(lambda :0)
for i in range(100000000):
    zz[1]=1

In [122]:
zz = dict({1:1})
for i in range(100000000):
    #zz.setdefault(1,1)
    zz[1]=zz[1]+1

In [56]:
b2b = blake2b()
b2b.update('catdog')
b2b.update(b'zzz')
b2b.hexdigest()

TypeError: Unicode-objects must be encoded before hashing

In [ ]:
zz = itty_b3hash(b'\xff\xff\xff\xff\xff\xff')
zzz = itty_b3hash(b'\xff\xff\xff\xaa\xaf\xaf')


In [ ]:
gbytes.similarity(zz,zzz)/64

In [ ]:
cats = b3hash('cat')
caats = b3hash('caats')
caaaats = b3hash('caaaats')
catscats = b3hash('catscats')
bcatscats = b3hash('bcatscats')
caattaats = b3hash('caattaats')


In [ ]:
elephant = b3hash('elephant')
eleephant = b3hash('eleephant')

In [ ]:
alice = b3hash('Alice was beginning to get very tired of sitting by her sister on the bank, and of having nothing to do')
april = b3hash('April was beginning to get very tired of sitting by her sister on the bank, and of having nothing to do')
april2 = b3hash('April was finally tired of sitting by her brother by the tree, and of having nothing to do')
ana = b3hash('of sitting by her nothing to do the bank, and of having Ana was beginning sister on to get very tired ')
#ana = b3hash('Ana was beginning to get very tired of sitting by her sister on the bank, and of having nothing to do')


long_sentence = b3hash('This is an extremely long sentence which should result in similarity for some reason ok bye')
longsentence = b3hash('Thisisanextremelylongsentencewhichshouldresultinsimilarityforsomereasonokbyesomemorewordshere')

shadowsfollow = b3hash('shadowsfollow')

In [ ]:
garbage = b3hash('qwertysjdfniabuioopkjhgfdfvbsdljfnjkasdnxcvbkxjcvdfkjnndmmbyasdkjfbkhbsdfkzxcvibekxcbvkasdljfnkjksbdf')


In [ ]:
pow = b3hash('aaaaab')
wow = b3hash('aaaaac')

In [ ]:
garbage2 = b3hash(str([random.randint(87,122) for _ in range(5)]))

In [ ]:
gbytes.similarity(pow,wow) / 256

In [ ]:
gbytes.similarity(alice,april) / 256

In [ ]:
gbytes.similarity(alice,april2) / 256

In [ ]:
gbytes.similarity(alice,ana) / 256

In [ ]:
gbytes.similarity(april,garbage2) / 256

In [ ]:
gbytes.similarity(long_sentence,april) / 256

In [ ]:
gbytes.similarity(long_sentence,alice) / 256

In [ ]:
gbytes.similarity(garbage,april) / 256

In [ ]:
gbytes.similarity(longsentence,long_sentence) / 256

In [ ]:
gbytes.similarity(garbage,longsentence) / 256

In [ ]:
gbytes.similarity(caattaats,shadowsfollow) / 256

In [ ]:
gbytes.similarity(garbage2,april) / 256

In [ ]:
garbage2

In [6]:
0xff//2+2

129

In [22]:
import numpy as np

192
37
71
71
71
71
71
71
71
71
71
71


<ipython-input-49-56ef47f041ce>:8: RuntimeWarning: overflow encountered in ulonglong_scalars
  wv = wv* np.uint64(0x8000000000000001)
<ipython-input-49-56ef47f041ce>:28: RuntimeWarning: overflow encountered in ulonglong_scalars
  pos = np.uint8(pos1+pos2)


[True,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False]

In [ ]:
8*32

In [2]:
import random
import numpy as np

In [3]:
x = np.ndarray(32,dtype=np.uint8)
x[0]  = np.uint8(0x57)
x[1]  = np.uint8(0x8d)
x[2]  = np.uint8(0xd1)
x[3]  = np.uint8(0xff)
x[4]  = np.uint8(0xad)
x[5]  = np.uint8(0x7f)
x[6]  = np.uint8(0x1a)
x[7]  = np.uint8(0x09)
x[8]  = np.uint8(0xfd)
x[9]  = np.uint8(0x65)
x[10] = np.uint8(0x61)
x[11] = np.uint8(0x52)
x[12] = np.uint8(0x18)
x[13] = np.uint8(0x93)
x[14] = np.uint8(0x01)
x[15] = np.uint8(0xe1)
x[16] = np.uint8(0x30)
x[17] = np.uint8(0x5d)
x[18] = np.uint8(0xd1)
x[19] = np.uint8(0xbf)
x[20] = np.uint8(0x17)
x[21] = np.uint8(0xcc)
x[22] = np.uint8(0x13)
x[23] = np.uint8(0xbd)
x[24] = np.uint8(0x29)
x[25] = np.uint8(0x0a)
x[26] = np.uint8(0xcf)
x[27] = np.uint8(0x31)
x[28] = np.uint8(0x98)
x[29] = np.uint8(0xc3)
x[30] = np.uint8(0x1a)
x[31] = np.uint8(0x0f)

In [96]:
for val in np.packbits(bitwheel):
    print(f'0x{val:02x}, ',end='')

0xf2, 0x3b, 0xf8, 0x76, 0xe6, 0x1a, 0x27, 0x3c, 0x32, 0x3d, 0x41, 0x4f, 0x45, 0x8d, 0x9b, 0x6a, 0xb4, 0xed, 0x5b, 0xa0, 0x55, 0x5e, 0x95, 0xf8, 0x5e, 0x59, 0x14, 0x60, 0xe1, 0x21, 0xb4, 0x74, 

In [6]:
wheel = np.array(list(range(256)),dtype=np.uint8)
np.random.shuffle(wheel)
bitwheel = np.array(list(range(256)),dtype=np.bool8)

bitwheel = np.random.choice(a=[True,False],size=256)
bitwheel.dtype = np.bool8


In [7]:
len(np.packbits(bitwheel)

32

In [89]:
wheel

array([231,  55, 152, 174,  16,  88,  14,  85,  42, 251,  40, 170,  32,
       141, 198, 133, 248,  89,  72, 161, 209,  98, 157, 229, 213,  92,
       186,  31,  96,  54, 204, 126, 202, 180,  29, 173,   7, 179,  34,
       148, 132, 242,  78,  37, 236, 167, 228, 165, 234,  87,  99, 214,
       109,   1,  62, 169,  22,   9, 118, 233, 223, 243, 195, 212, 124,
       255, 222, 215, 136, 177,  68, 142,  36, 115,   3,  67, 125, 137,
       146, 206, 121, 221,  11, 201, 239, 154,  81,  82, 252, 210, 123,
        45, 187, 247, 160, 112,  47, 225, 104, 217, 149, 111, 196, 171,
       240, 131,  39,  49, 119,  93, 128, 216,  63,  53,  41, 156, 194,
        25,  70, 150,  65,  17, 232,  38, 237,  35, 106,  33, 113,  69,
         5,  84,  43, 129, 107,  27,  73,  44,  52, 208, 185, 250, 191,
        76, 182, 116, 245, 230, 200,  61, 122,  77,  66,  97,  13,  51,
       108, 130, 120,   0, 110, 175, 166,  15, 102,   6, 184, 159, 162,
        56, 197, 238, 224, 254, 135, 227,  71, 101,  10, 188, 22

In [88]:
hex(wheel[0x98])

'0x42'

In [129]:
def sim(v1,v2):
    out = np.sum(np.bitwise_not(np.bitwise_xor(v1,v2)))
    return(out)

def thingy(binseed):
    pos = x.copy()
    last = np.uint8(0)
    for i in range(len(binseed)):
        if binseed[i] == True:
            
            for j in range(len(pos)):
                #if j == 4:
                #    print(f'{i}: pos before {pos[j]:02x}')
                #    print(f'{i}: wheel {wheel[pos[j]]:02x}')
                pos[j] += np.bitwise_xor(np.bitwise_xor(pos[j],wheel[pos[j]]),last) + np.uint8(i)
                last = pos[j]

        else:
            for j in range(len(pos)):
                #if j == 4:
                #    print(f'{i}: pos before {pos[j]:02x}')
                #    print(f'{i}: wheel {wheel[pos[j]]:02x}')
                pos[j] -= np.bitwise_xor(np.bitwise_xor(pos[j],wheel[pos[j]]),last) + np.uint8(i)
                last = pos[j]
        #print(f'{i}: pos after {pos[4]:02x}')
        #print(pos[j])
    output = np.ndarray(256,dtype=np.bool8)
    for i in range(256):
        bit_index = np.uint8(0)
        for j in range(len(pos)):
            bit_index += wheel[pos[j]]
            pos[j] += np.uint8(1)
        #print(f'{i}: {bit_index}')

        bit = bitwheel[bit_index]
        output[i] = bit
    return(output)

control_seed = np.random.choice(a=[True,False],size=2046)
randchange = np.random.randint(0,len(control_seed))
#print(control_seed[randchange])
test_seed = control_seed.copy()
test_seed[randchange] = np.bitwise_not(test_seed[randchange])
#print(test_seed[randchange])
#print(control_seed[randchange])
control1 = thingy(control_seed)
#print('---------------------')
test1 = thingy(test_seed)
#print('---------------------')
similarity = sim(control1,test1)
print(similarity)
#if abs(similarity) > 30:
#    print(i)
#    print(similarity)
#    break
#        

<ipython-input-129-560f86b9a5b5>:23: RuntimeWarning: overflow encountered in ubyte_scalars
  pos[j] -= np.bitwise_xor(np.bitwise_xor(pos[j],wheel[pos[j]]),last) + np.uint8(i)
<ipython-input-129-560f86b9a5b5>:15: RuntimeWarning: overflow encountered in ubyte_scalars
  pos[j] += np.bitwise_xor(np.bitwise_xor(pos[j],wheel[pos[j]]),last) + np.uint8(i)
<ipython-input-129-560f86b9a5b5>:31: RuntimeWarning: overflow encountered in ubyte_scalars
  bit_index += wheel[pos[j]]
<ipython-input-129-560f86b9a5b5>:32: RuntimeWarning: overflow encountered in ubyte_scalars
  pos[j] += np.uint8(1)


136


In [15]:
for i in np.packbits(test_seed):
    print(f'{i}, ',end='')

81, 172, 237, 165, 218, 77, 113, 30, 62, 96, 154, 191, 243, 248, 172, 146, 212, 96, 157, 177, 101, 50, 0, 222, 187, 115, 63, 63, 49, 151, 139, 127, 

In [44]:
0x57 - ((0x57 ^ 0x52) +0)

82

In [98]:
116-128

-12